In [ ]:
!pip3 install num2words # Install library untuk mengubah angka menjadi kata.
!pip install Sastrawi # Karena wordnet tidak bisa digunakan maka di sini pakai Sastrawi.

^C
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from num2words import num2words
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import tensorflow as tf
from tensorflow import keras
import os

nltk.download('punkt') # Untuk tokenizer kata.
nltk.download('stopwords')# Untuk menghapus stopwords.

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('/kaggle/input/model-project-v1/model-project-v1.h5')

# Show the model architecture
new_model.summary()

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

def preprocess_text(text):
    text = word_tokenize(text.lower()) # Tokenize text_id ke dalam token/kata
    text = [t for t in text if t not in stop_words] # Hapus stop_words
    text = [stemmer.stem(t) for t in text] # Menjadikan kata dasar
#     text = [t if not t.isdigit() else num2words(int(t)) for t in text] # Mengganti angka ke teks
    text = ' '.join(text) # Gabungkan ke dalam teks kembali
    return text

In [ ]:
pd.set_option('display.max_colwidth', None)

## Data Depresi

In [ ]:
data = pd.DataFrame([
"Setiap hari, aku hanya berpikir untuk bunuh diri."], columns=['text'])

# data = pd.DataFrame([
# "Aku merasa benar-benar tak berarti dan tidak ada yang peduli dengan keberadaanku."], columns=['text'])

In [ ]:
data['nlp_text'] = data['text'].apply(lambda x: preprocess_text(x))

In [ ]:
data

In [ ]:
words = set(word for sentence in data["nlp_text"] for word in sentence.split())
vocab_size = len(words)
print(vocab_size)

In [ ]:
one_hot_representation = [one_hot(words,vocab_size) for words in data["nlp_text"]]

In [ ]:
embedded_docs = pad_sequences(one_hot_representation, padding = 'post',truncating = 'post', maxlen = 44)

In [ ]:
y_pred = new_model.predict(embedded_docs)
y_pred

In [ ]:
y_pred = (y_pred >= 0.5).astype("int")
if(y_pred==0):
    print("tidak depresi")
elif(y_pred==1):
    print("depresi")